In [1]:
import os
from math import floor
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import yaml
import PIL

In [2]:
DATA_PATH = os.path.join("..", 'data', 'vedai_corrected', 'images')
RELATIVE_PATH = os.listdir(DATA_PATH)
ABS_PATH = [os.path.join(DATA_PATH, file_path) for file_path in RELATIVE_PATH]
TEST_IMG_PATH = ABS_PATH[100]
# TEST_IMG_PATH = "test_img.png"

In [3]:
batch_size = 32
img_height = 1024
img_width = 1024

def decode_img(img_path, convert_dtype=True):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.read_file(img_path)
  img = tf.io.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, [img_height, img_width])
  if convert_dtype:
    img = tf.image.convert_image_dtype(img, dtype=tf.uint8, saturate=False)
  img = tf.reshape(img, [1, img_height, img_width, 3])
  # Resize the image to the desired size
  return img

def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color='red',
                               thickness=4,
                               display_str_list=(),
                               use_normalized_coordinates=True):
  """Adds a bounding box to an image.
  Bounding box coordinates can be specified in either absolute (pixel) or
  normalized coordinates by setting the use_normalized_coordinates argument.
  Each string in display_str_list is displayed on a separate line above the
  bounding box in black text on a rectangle filled with the input 'color'.
  If the top of the bounding box extends to the edge of the image, the strings
  are displayed below the bounding box.
  Args:
    image: a PIL.Image object.
    ymin: ymin of bounding box.
    xmin: xmin of bounding box.
    ymax: ymax of bounding box.
    xmax: xmax of bounding box.
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list: list of strings to display in box
                      (each to be shown on its own line).
    use_normalized_coordinates: If True (default), treat coordinates
      ymin, xmin, ymax, xmax as relative to the image.  Otherwise treat
      coordinates as absolute.
  """
  draw = PIL.ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  if thickness > 0:
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
               (left, top)],
              width=thickness,
              fill=color)
  try:
    font = PIL.ImageFont.truetype('arial.ttf', 24)
  except IOError:
    font = PIL.ImageFont.load_default()

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getbbox(ds)[3] - font.getbbox(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    str_left, str_top, str_right, str_bottom = font.getbbox(display_str)
    text_width, text_height = str_bottom - str_top, str_right - str_left
    margin = np.ceil(0.05 * text_height)
    draw.rectangle(
        [(left, text_bottom - text_height - 2 * margin), (left + text_width,
                                                          text_bottom)],
        fill=color)
    draw.text(
        (left + margin, text_bottom - text_height - margin),
        display_str,
        fill='black',
        font=font)
    text_bottom -= text_height - 2 * margin


def draw_bounding_boxes_on_image(image,
                                 boxes,
                                 color='red',
                                 thickness=4,
                                 display_str_list_list=()):
  """Draws bounding boxes on image.
  Args:
    image: a PIL.Image object.
    boxes: a 2 dimensional numpy array of [N, 4]: (ymin, xmin, ymax, xmax). The
      coordinates are in normalized format between [0, 1].
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list_list: list of list of strings. a list of strings for each
      bounding box. The reason to pass a list of strings for a bounding box is
      that it might contain multiple labels.
  Raises:
    ValueError: if boxes is not a [N, 4] array
  """
  boxes_shape = boxes.shape
  if not boxes_shape:
    return
  if len(boxes_shape) != 2 or boxes_shape[1] != 4:
    raise ValueError('Input must be of size [N, 4]')
  for i in range(boxes_shape[0]):
    display_str_list = ()
    if display_str_list_list:
      display_str_list = display_str_list_list[i]
    draw_bounding_box_on_image(image, boxes[i, 0], boxes[i, 1], boxes[i, 2],
                               boxes[i, 3], color, thickness, display_str_list)

def create_nms_bbox(test_img, detector_output):
  suppressed_idx = tf.image.non_max_suppression(
    tf.squeeze(detector_output["detection_boxes"]),
    tf.squeeze(detector_output["detection_scores"]),
    max_output_size=tf.constant(500),
    iou_threshold=0.3,
    score_threshold=float('-inf'),
    name=None
)

  bounding_boxes_suppressed = tf.gather(tf.squeeze(detector_output["detection_boxes"]), suppressed_idx).numpy().squeeze()
  labels_suppressed = tf.gather(tf.squeeze(detector_output["detection_classes"]), suppressed_idx).numpy().astype(int).astype(str).squeeze().tolist()
  draw_bounding_boxes_on_image(test_img, bounding_boxes_suppressed, color='red', thickness=4, display_str_list_list=labels_suppressed)
  return test_img

# VEDAI Dataset creation

In [11]:

DRIVE_ROOT = os.path.join("..")
CORRECTED_PATH = os.path.join(DRIVE_ROOT, "data", "vedai_corrected")
ANNOTATIONS_DIR = os.path.join(CORRECTED_PATH, "annotations")
ANNOTATIONS_MERGED_FILE_PATH = os.path.join(ANNOTATIONS_DIR, "annotations.csv")
IMAGES_DIR = os.path.join(CORRECTED_PATH, "images")
CLASSES_DICT = {1: "car",
                2: "truck",
                3: "pickup",
                4: "tractor",
                5: "camping car",
                6: "boat",
                7: "motorcycle",
                8: "bus",
                9: "van",
                10: "other",
                11: "small plane",
                12: "large plane"}

def merge_annotation_folder(annotations_dir, classes_dict, train_proportion=0.8, test_proportion=0.1):
  if train_proportion+test_proportion>=1:
    raise ValueError("Input a train and test proportion summing up to strictly less than 1.")
  annotation_files = list(Path(annotations_dir).rglob('*.txt'))
  indices = list(range(len(annotation_files)))
  files = [str(idx) + '.txt' for idx in indices]
  # else:
  #   files = [filename for filename in list(sorted(os.listdir(self.annotations_dir))) if filename.endswith('.txt')]
  #   indices = [convert_id_to_idx(filename.replace('.txt', '')) for filename in files]
  abs_filepaths = [os.path.join(annotations_dir, annotation_file) for annotation_file in files]
  annotations = pd.DataFrame(columns=["dataset", "x", "y", "width", "length", "idx", "empty"])
  # for img_file, filepath in zip(files, abs_filepaths):
  for idx, img_file in zip(indices, abs_filepaths):
    temp_annotation = pd.read_csv(img_file, sep=' ', names=["x", "y", "width", "length"]).reset_index(drop=False)
    # temp_annotation["image_id"] = img_file.split('.')[0]
    temp_annotation["idx"] = idx
    annotations = pd.concat([annotations, temp_annotation])
  annotations = annotations.rename(columns={"index":"labels"})
  annotations["labels"] = (annotations["labels"] + 1).astype(int)
  # annotations.index.name = None
  annotations["labels_name"] = annotations["labels"].replace(classes_dict)
  annotations["x_min"] = (annotations["x"] - annotations["width"]/2)
  annotations["y_min"] = (annotations["y"] - annotations["length"]/2)
  annotations["x_max"] = (annotations["x"] + annotations["width"]/2)
  annotations["y_max"] = (annotations["y"] + annotations["length"]/2)
  annotations["image_path"] = annotations["idx"].astype(str) + ".jpg"
  # Train test split
  images_idx = annotations.index.unique()
  train_idx = np.random.choice(images_idx, replace=False, size=floor(train_proportion*len(images_idx)))
  test_idx = np.random.choice(list(set(images_idx) - set(train_idx)), replace=False, size=floor(test_proportion*len(images_idx)))
  validation_idx = list(set(images_idx) - set(train_idx) - set(test_idx))
  annotations.loc[train_idx, "dataset"] = "TRAIN"
  annotations.loc[test_idx, "dataset"] = "TEST"
  annotations.loc[validation_idx, "dataset"] = "VALIDATION"
  annotations = annotations[["dataset", "image_path", "labels_name", "x_min", "y_min", "empty", "empty", "x_max", "y_max", "empty", "empty"]]
  return annotations

if not os.path.exists(ANNOTATIONS_MERGED_FILE_PATH):
  annotations = merge_annotation_folder(ANNOTATIONS_DIR, CLASSES_DICT, train_proportion=0.8, test_proportion=0.1)
  annotations.to_csv(os.path.join(ANNOTATIONS_DIR, "annotations.csv"), index=False, header=False)

In [ ]:
# def create_tf_example(example):
#   # TODO(user): Populate the following variables from your example.
#   height = None # Image height
#   width = None # Image width
#   filename = None # Filename of the image. Empty if image is not from file
#   encoded_image_data = None # Encoded image bytes
#   image_format = None # b'jpeg' or b'png'

#   xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
#   xmaxs = [] # List of normalized right x coordinates in bounding box
#              # (1 per box)
#   ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
#   ymaxs = [] # List of normalized bottom y coordinates in bounding box
#              # (1 per box)
#   classes_text = [] # List of string class name of bounding box (1 per box)
#   classes = [] # List of integer class id of bounding box (1 per box)

#   tf_example = tf.train.Example(features=tf.train.Features(feature={
#       'image/height': dataset_util.int64_feature(height),
#       'image/width': dataset_util.int64_feature(width),
#       'image/filename': dataset_util.bytes_feature(filename),
#       'image/source_id': dataset_util.bytes_feature(filename),
#       'image/encoded': dataset_util.bytes_feature(encoded_image_data),
#       'image/format': dataset_util.bytes_feature(image_format),
#       'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
#       'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
#       'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
#       'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
#       'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
#       'image/object/class/label': dataset_util.int64_list_feature(classes),
#   }))
#   return tf_example

# TF Lite - Detection Module

In [8]:
tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [9]:
spec = model_spec.get('efficientdet_lite4')

In [12]:
# TO-DO : generate TFRecord files to accelerate loading
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(os.path.join(ANNOTATIONS_DIR, 'annotations.csv'), images_dir=IMAGES_DIR)

In [13]:
train_data

In [2]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=False, validation_data=validation_data)

NameError: name 'object_detector' is not defined

In [ ]:
model.evaluate(test_data)

# TF Classical

In [ ]:
tf_image = decode_img(TEST_IMG_PATH)
image = ...  # A batch of preprocessed images with shape [batch_size, height, width, 3].
base_model = hub.KerasLayer("https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/1")
cls_outputs, box_outputs = base_model(image, training=training)

In [ ]:

detector_output = efficient_det_2(tf_image)

test_img = PIL.Image.open(TEST_IMG_PATH)

test_img = create_nms_bbox(test_img, detector_output)
test_img

In [54]:
m = tf.keras.Sequential([hub.KerasLayer("efficientdet_d6_1", trainable=True),
    tf.keras.layers.Dense(13, activation='softmax')
])

In [55]:
m.build([1, 1024, 1024, 3])
m.summary()

ERROR:absl:hub.KerasLayer is trainable but has zero trainable weights.


ValueError: Exception encountered when calling layer "keras_layer" "                 f"(type KerasLayer).

in user code:

    File "c:\Users\berto\mambaforge\envs\satellite\lib\site-packages\tensorflow_hub\keras_layer.py", line 229, in call  *
        result = f()

    ValueError: Python inputs incompatible with input_signature:
      inputs: (
        Tensor("Placeholder:0", shape=(1, 1024, 1024, 3), dtype=float32))
      input_signature: (
        TensorSpec(shape=(1, None, None, 3), dtype=tf.uint8, name=None)).


Call arguments received by layer "keras_layer" "                 f"(type KerasLayer):
  • inputs=tf.Tensor(shape=(1, 1024, 1024, 3), dtype=float32)
  • training=None

In [3]:
efficient_det_2 = hub.load("https://tfhub.dev/tensorflow/efficientdet/d6/1")

In [49]:
tf.keras.utils.plot_model(
    efficient_det_2,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
)

AttributeError: '_UserObject' object has no attribute 'built'

In [6]:
# model_test = tf.load("detection_retinanet_spinenet-96.tar")
with open("spinenet96_retinanet.yaml", "r") as file:
    model_config = yaml.load(file, Loader=yaml.FullLoader)